# Evaluation Per Topic on Robust04

### Import utility and load datasets

In [1]:
%%time
import sys
sys.path.append('../python/')
from tqdm import tqdm
from glob import glob
import pandas as pd
from parametrized_bootstrapping_model import ParametrizedBootstrappingModel, ReturnAlways1Model, ReturnAlways0Model
from result_analysis_utils import load_ground_truth_data, load_evaluations, run_cross_validation, load_cross_validation_results, load_raw_evaluations
SEARCH_SPACE= [0, 1, 2] + list(range(5,96, 5)) + [98, 99, 100]
from io import StringIO
from trectools import TrecQrel
from sklearn.metrics import mean_squared_error
import numpy as np
import json


if 'df' not in locals() or 'unique_queries' not in locals():
    eval_predictions = glob('../resources/eval/trec-system-runs/trec13/*.jsonl')
    eval_predictions += list(load_cross_validation_results(open('tmp-results.jsonl'), depth=10, return_buffers=True))
    
    df = load_evaluations(tqdm(eval_predictions))
    
    unique_queries = set(TrecQrel('../resources/unprocessed/topics-and-qrels/qrels.robust04.txt').qrels_data['query'].astype(str).unique())

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2302/2302 [01:14<00:00, 30.93it/s]


CPU times: user 1min 39s, sys: 6.86 s, total: 1min 46s
Wall time: 1min 47s


In [2]:
df

,,"(depth-10-complete, residual-ndcg@20-min)","(depth-10-complete, residual-ndcg@20-max)","(depth-10-incomplete, residual-ndcg@20-min)","(depth-10-incomplete, residual-ndcg@20-max)","(depth-20-complete, residual-ndcg@20-min)","(depth-20-complete, residual-ndcg@20-max)","(depth-20-incomplete, residual-ndcg@20-min)","(depth-20-incomplete, residual-ndcg@20-max)","(depth-10-complete, condensed-ndcg@10)","(depth-10-incomplete, condensed-ndcg@10)",...,"(depth-20-complete, unjudged@20)","(depth-20-incomplete, unjudged@20)","(depth-10-incomplete, pbs-rmse[0.6,1]-bs-p-1000-ndcg@10-ndcg@10)","(depth-10-incomplete, pbs-rmse[0.8,1]-bs-p-1000-ndcg@10-ndcg@10)","(depth-10-incomplete, pbs-rmse-bs-p-1000-ndcg@10-ndcg@10)","(depth-10-incomplete, pbs-rmse[1,2]-bs-p-1000-ndcg@10-ndcg@10)","(depth-10-incomplete, pbs-rmse[1,3]-bs-p-1000-ndcg@10-ndcg@10)","(depth-10-incomplete, always-1)","(depth-10-incomplete, always-0)",run
run,,,,,,,,,,,,,,,,,,,,,,
JuruDes,0,"{""301"": 0.35236452400022594, ""302"": 0.64323925...","{""301"": 0.4303702292214245, ""302"": 0.643239257...","{""301"": 0.21032448436656867, ""302"": 0.64323925...","{""301"": 0.4777169090993102, ""302"": 0.643239257...","{""301"": 0.35236452400022594, ""302"": 0.64323925...","{""301"": 0.39074916216560884, ""302"": 0.64323925...","{""301"": 0.35236452400022594, ""302"": 0.64323925...","{""301"": 0.4380958420434946, ""302"": 0.643239257...","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0.12987501165334076, ""302"": 0.77215962...",...,"{""301"": 0.05, ""302"": 0.0, ""303"": 0.0, ""304"": 0...","{""301"": 0.1, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.0, ""302"": 0.7721596255000001, ""303"":...","{""301"": 0.0, ""302"": 0.7721596255000001, ""303"":...","{""301"": 0.0, ""302"": 0.7721596255000001, ""303"":...","{""301"": 0.0733639221, ""302"": 0.772159625500000...","{""301"": 0.0733639221, ""302"": 0.772159625500000...","{""301"": 1.0, ""302"": 1.0, ""303"": 1.0, ""308"": 1....","{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""308"": 0....",JuruDes
JuruDesAggr,0,"{""301"": 0.3297722027615143, ""302"": 0.643239257...","{""301"": 0.3661284907500531, ""302"": 0.643239257...","{""301"": 0.27482361468952843, ""302"": 0.64323925...","{""301"": 0.4088868032650847, ""302"": 0.643239257...","{""301"": 0.3297722027615143, ""302"": 0.643239257...","{""301"": 0.3661284907500531, ""302"": 0.643239257...","{""301"": 0.3297722027615143, ""302"": 0.643239257...","{""301"": 0.4088868032650847, ""302"": 0.643239257...","{""301"": 0.3557772116892465, ""302"": 0.772159625...","{""301"": 0.2180164145324244, ""302"": 0.772159625...",...,"{""301"": 0.05, ""302"": 0.0, ""303"": 0.0, ""304"": 0...","{""301"": 0.1, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.1517621911, ""302"": 0.772159625500000...","{""301"": 0.1517621911, ""302"": 0.772159625500000...","{""301"": 0.1517621911, ""302"": 0.772159625500000...","{""301"": 0.1517621911, ""302"": 0.772159625500000...","{""301"": 0.2180164145, ""302"": 0.772159625500000...","{""301"": 1.0, ""302"": 1.0, ""303"": 1.0, ""308"": 1....","{""301"": 0.0, ""302"": 0.0, ""303"": 0.0, ""308"": 0....",JuruDesAggr
JuruDesLaMd,0,"{""301"": 0.35236452400022594, ""302"": 0.64220566...","{""301"": 0.4303702292214245, ""302"": 0.642205662...","{""301"": 0.21032448436656867, ""302"": 0.64220566...","{""301"": 0.4777169090993102, ""302"": 0.642205662...","{""301"": 0.35236452400022594, ""302"": 0.64220566...","{""301"": 0.39074916216560884, ""302"": 0.64220566...","{""301"": 0.35236452400022594, ""302"": 0.64220566...","{""301"": 0.4380958420434946, ""302"": 0.642205662...","{""301"": 0.22009176629808017, ""302"": 0.77215962...","{""301"": 0.12987501165334076, ""302"": 0.77215962...",...,"{""301"": 0.05, ""302"": 0.0, ""303"": 0.0, ""304"": 0...","{""301"": 0.1, ""302"": 0.0, ""303"": 0.0, ""304"": 0....","{""301"": 0.0, ""302"": 0.7721596255000001, ""303"":...","{""301"": 0.0, ""302"": 0.7721596255000001, ""303"":...","{""301"": 0.0, ""302"":

In [13]:
def report_for_row(df_row, measure, depth):
    tmp = {'run': df_row['run'].split('/')[-1].replace('input.', '').replace('.gz', '')}
    measures = [
        ('unjudged', (f'depth-{depth}-incomplete', f'unjudged@{depth}')),
        (f'ground-truth-{measure}@{depth}', (f'depth-{depth}-complete', f'ndcg@{depth}')),
        (f'min-residual-{measure}@{depth}', (f'depth-{depth}-incomplete', f'residual-{measure}@{depth}-min')),
        (f'condensed-{measure}@{depth}', (f'depth-{depth}-incomplete', f'condensed-{measure}@{depth}')),
        (f'max-residual-{measure}@{depth}', (f'depth-{depth}-incomplete', f'residual-{measure}@{depth}-max')),
        (f'always-1', (f'depth-{depth}-incomplete', 'always-1')),
        (f'always-0', (f'depth-{depth}-incomplete', 'always-0')),
        
        # Bootstrapping
        (f'PBS-RMSE[0.6,1]-{measure}@{depth}', (f'depth-{depth}-incomplete', f'pbs-rmse[0.6,1]-bs-p-1000-{measure}@{depth}-{measure}@{depth}')),
        (f'PBS-RMSE[0.8,1]-{measure}@{depth}', (f'depth-{depth}-incomplete', f'pbs-rmse[0.8,1]-bs-p-1000-{measure}@{depth}-{measure}@{depth}')),
        (f'PBS-RMSE-{measure}@{depth}', (f'depth-{depth}-incomplete', f'pbs-rmse-bs-p-1000-{measure}@{depth}-{measure}@{depth}')),
        (f'PBS-RMSE[1,2]-{measure}@{depth}', (f'depth-{depth}-incomplete', f'pbs-rmse[1,2]-bs-p-1000-{measure}@{depth}-{measure}@{depth}')),
        (f'PBS-RMSE[1,3]-{measure}@{depth}', (f'depth-{depth}-incomplete', f'pbs-rmse[1,3]-bs-p-1000-{measure}@{depth}-{measure}@{depth}')),
    ]
    
    for display_name, m in measures:
        tmp[display_name] = json.loads(df_row[m])
    
    ret = []
    
    for topic in tmp[f'ground-truth-{measure}@{depth}']:
        entry = {'run': tmp['run'], 'topic': topic}
        for k, v in tmp.items():
            if k in ['run']:
                continue
            
            if topic in v:
                entry[k] = v[topic]
        ret += [entry]
    
    return ret

def create_aggregated_df(measure, depth, loc):
    ret = pd.DataFrame([dict(i) for i in report_for_row(df.iloc[loc], measure, depth)])
    ret = ret.sort_values(f'ground-truth-{measure}@{depth}', ascending=False).reset_index()
    del ret['index']
    return ret

def rmse():
    ret = []
    for run in tqdm(range(110)):
        try:
            tmp = create_aggregated_df('ndcg', 10, run)
        except:
            continue
        tmp = tmp[tmp['unjudged'] > 0].dropna()
        if len(tmp) < 50:
            continue

        for k,v in [('Condensed', 'condensed-ndcg@10'), ('Min-Residual', 'min-residual-ndcg@10'),
                    ('Max-Residual', 'max-residual-ndcg@10'), ('Always 1', 'always-1'), ('Always 0', 'always-0'),
                   
                    ('PBS-RMSE[0.6,1]', 'PBS-RMSE[0.6,1]-ndcg@10'),
                    ('PBS-RMSE[0.8,1]', 'PBS-RMSE[0.8,1]-ndcg@10'),
                    ('PBS-RMSE', 'PBS-RMSE-ndcg@10'),
                    ('PBS-RMSE[1,2]', 'PBS-RMSE[1,2]-ndcg@10'),
                    ('PBS-RMSE[1,3]', 'PBS-RMSE[1,3]-ndcg@10')
                   ]:
            ground_truth = tmp['ground-truth-ndcg@10']
            comp = tmp[v]
            
            ret += [{'Method': k,
                     'RMSE (All)': mean_squared_error(ground_truth, comp, squared=False),
                     'RMSE (Underestimation)': mean_squared_error(ground_truth, np.minimum(comp, ground_truth), squared=False),
                     'RMSE (Overestimation)': mean_squared_error(ground_truth, np.maximum(comp, ground_truth), squared=False),
                    }]
            
    
    return pd.DataFrame(ret)

df_rmse = rmse().groupby('Method').aggregate({'RMSE (All)': 'mean', 'RMSE (Overestimation)': 'mean', 'RMSE (Underestimation)': 'mean', }).reset_index()
df_rmse.sort_values('RMSE (All)')


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 110/110 [00:00<00:00, 127.74it/s]


,Method,RMSE (All),RMSE (Overestimation),RMSE (Underestimation)
5,PBS-RMSE,0.055344,0.019938,0.050748
4,Min-Residual,0.055355,0.003719,0.055076
7,"PBS-RMSE[0.8,1]",0.055643,0.010851,0.053560
6,"PBS-RMSE[0.6,1]",0.056291,0.016830,0.051149
8,"PBS-RMSE[1,2]",0.062723,0.030963,0.047645
2,Condensed,0.075653,0.070348,0.025602
9,"PBS-RMSE[1,3]",0.088212,0.074771,0.036778
3,Max-Residual,0.229156,0.229153,0.000348
0,Always 0,0.464755,0.000000,0.464755
1,Always 1,0.690333,0.690333,0.000000
